In [ ]:
#import statements
import joblib
import numpy as np
import networkx as nx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import pyplot as plt


from matplotlib import rcParams
#rcParams['font.family'] = 'Helvetica'

In [ ]:
pip install ruptures

In [ ]:
#load the data and convert to an array
data = (r"C:\Users\aditi\OneDrive - Georgia State University\UAP math\hup txt and pkl files\Patient88_01Clean.txt")

x = np.loadtxt(data);

In [ ]:
plt.figure(figsize = (6,4), dpi = 600)
fs = 512
window = fs * 8
shift = fs * 4
print(x.shape[0])
time = x.shape[0]/fs
print(time)
time_vector = np.linspace(0, time, x.shape[0])
plt.plot(time_vector, x)
plt.ylabel('Amplitude')
plt.xlabel('time (s)')
plt.show()

In [ ]:
x_mean = np.mean(x, axis=1)
print(f"Avg Time Series Shape: {x_mean.shape}")

plt.plot(time_vector, x_mean)
plt.show()

In [ ]:
plt.figure(figsize=(12,5), dpi = 300)

x_mean = np.mean(x, axis=1)
print(f"Avg Time Series Shape: {x_mean.shape}")

dx_mean = np.gradient(x_mean)
dx2_mean = np.gradient(dx_mean)

plt.plot(time_vector, x_mean, color="blue")
plt.plot(time_vector, dx_mean, color="red")
plt.plot(time_vector, dx2_mean, color="black")
plt.title("Patient 88")

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Assume 'x' is your time series (timepoints, channels)
x_mean = np.mean(x, axis=1)  # Shape: (timepoints,)
# Normalize the time series (Optional, improves detection)
x_mean = (x_mean - np.min(x_mean)) / (np.max(x_mean) - np.min(x_mean))
# Create time vector (adjust Fs based on your dataset)
Fs = 512  # Example: Sampling rate in Hz
t = np.arange(len(x_mean)) / Fs  # Time vector in seconds

In [ ]:
import ruptures as rpt
# Change point detection using the Pelt algorithm (Dynamic Programming)
algo = rpt.Pelt(model="l2", min_size=100, jump=100).fit(x_mean)
change_points = algo.predict(pen=10)  # Adjust 'pen' for sensitivity
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(t, x_mean, label="Averaged Time Series", color="blue")
plt.ylabel("Amplitude")
plt.xlabel("Time (seconds)")
plt.title("Change Point Detection in Epileptic Time Series")
# Mark detected change points
for cp in change_points:
    plt.axvline(t[cp], color="red", linestyle="--", label="Detected Change Point" if cp == change_points[0] else "")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4), dpi=600)  # Adjusted figsize for better visualization
variance = np.var(x, axis=1),
time = x.shape[0] / fs
print("Total time:", time)
time_vector = np.linspace(0, time, x.shape[0])

# Plot variance
plt.plot(time_vector, variance)
plt.xticks(np.arange(0, time, 3), fontsize = 8)  

plt.xlabel('Time (s)')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
x.shape

In [ ]:
fs = 512
window = fs * 8  
shift = fs * 4
print("window" , window) # No of points
print("shift" , shift)   # No of points

In [ ]:
length = x.shape[0]
print(length)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fs = 512
window_sec = 8     # window length in seconds
shift_sec = 4      # shift length in seconds

length = x.shape[0] / fs   # length in seconds
print('length', length)

# Calculate the number of networks (windows)
num_win = (length - window_sec) / shift_sec + 1
print('num_win', num_win)

num_networks = int(np.ceil(num_win))
print('num_networks', num_networks)

# Initialize list for network times
network_times = []

# Loop through each network/window
for i in range(num_networks):
    # Calculate the start time of the current window in seconds
    start_time = i * shift_sec
    # Calculate the midpoint time of the current window in seconds
    midpoint_time = start_time + (window_sec / 2)
    # Append the midpoint time to the list
    network_times.append(midpoint_time)

# Convert the list to a numpy array for easy manipulation and display
network_times = np.array(network_times)

# Print the vector of network times
print("Network times (in seconds):", network_times)

In [ ]:
len(network_times)

In [ ]:
np.where(network_times == 268) # Onset

In [ ]:
np.where(network_times == 340) # offset at 338 seconds

In [ ]:
#load data
results88_8s = joblib.load(r"C:\Users\aditi\OneDrive - Georgia State University\UAP math\hup txt and pkl files\results88_data8s4s.pkl")

In [ ]:
from tqdm import tqdm
from matplotlib.ticker import FuncFormatter
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from matplotlib import gridspec

x = x
networks = results88_8s

c1 = np.asarray(networks)
As = []
binarized_networks = []
in_degree = np.zeros((c1.shape[0], x.shape[1]))
out_degree = np.zeros((c1.shape[0], x.shape[1]))
degree = np.zeros((c1.shape[0], x.shape[1]))
for i, G in enumerate(tqdm(networks)):
    A = nx.adjacency_matrix(G)
    A = A.todense()
    As.append(A)
    crit = np.percentile(np.array(A).flatten(), [0, 95])[1] #np.percentile(A, [2.75, 85])[1]
    A[A < crit] = 0
    A[A >= crit] = 1
    G = nx.from_numpy_array(A, create_using=nx.DiGraph())
    binarized_networks.append(G)
    for j, node in enumerate(G.nodes):
        in_degree[i,j] = G.in_degree(node)
        out_degree[i,j] = G.out_degree(node)
        degree[i,j] = G.degree(node)

# Lists to store the calculated features
avg_clustering_coeffs = []
transitivities = []

# Calculate the features for each network
for G in binarized_networks:
    # Average clustering coefficient
    avg_clustering_coeffs.append(nx.average_clustering(G))
    
    # Transitivity
    transitivities.append(nx.transitivity(G))

    
# Initialize an empty list to store betweenness centrality values for each digraph
betweenness_centrality_list = []

# Iterate through the digraphs
for G in binarized_networks:
    # Compute betweenness centrality for the current digraph
    betweenness_dict = nx.betweenness_centrality(G)
    
    # Get the number of nodes in the current digraph
    num_nodes = len(G.nodes)
    
    # Initialize an array of zeros with the size equal to the number of nodes
    betweenness_array = np.zeros(num_nodes)
    
    # Iterate through the betweenness_dict and store the centrality values in the array
    for node, centrality in betweenness_dict.items():
        betweenness_array[node] = centrality
        
    # Append the betweenness_array to the betweenness_centrality_list
    betweenness_centrality_list.append(betweenness_array)

# Convert the betweenness_centrality_list to a NumPy array
betweenness_centrality = np.array(betweenness_centrality_list)

# Calculate the average betweenness centrality for each graph
avg_betweenness_centralities = np.mean(betweenness_centrality, axis=1)

In [ ]:

# Set the Seaborn theme
sns.set_theme(style="darkgrid", font_scale=1.2)

# # Define the figure
#fig = plt.figure(figsize=(35, 25))

fig = plt.figure(figsize=(25, 25))

# # Create a GridSpec with 15 rows and 2 columns
#gs = gridspec.GridSpec(15, 2)
gs = gridspec.GridSpec(15, 1)

# Allocate grid sizes for the first column
ax5 = plt.subplot(gs[0:3, 0])   # Time series plot
ax4 = plt.subplot(gs[3:6, 0])   # Topological features
ax1 = plt.subplot(gs[6:9, 0])   # Derivative of Degree
ax2 = plt.subplot(gs[9:12, 0])  # Derivative of Outdegree
ax3 = plt.subplot(gs[12:15, 0]) # Derivative of Betweenness Centrality


# Subplot One

yd = np.diff(degree.T) #np.diff(out_degree.T)
z_min, z_max = yd.min(), np.abs(yd).max()#-np.abs(yd).max(), np.abs(yd).max()
#z_min, z_max = -np.abs(yd).max(), np.abs(yd).max()

c = ax1.pcolormesh(yd, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("bottom", size="5%", pad=1.5)
fig.colorbar(c, cax=cax, orientation='horizontal')
#fig.colorbar(c, orientation='horizontal')


ax1.set_xlabel('Time (s)')
ax1.set_title("Derivative of Degree")


# Subplot Two

yd = np.diff(out_degree.T)

z_min, z_max = -np.abs(yd).max(), np.abs(yd).max()

c = ax2.pcolormesh(yd, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
divider = make_axes_locatable(ax2)
cax = divider.append_axes("bottom", size="5%", pad=1.5)
fig.colorbar(c, cax=cax, orientation='horizontal')
ax2.set_title("Derivative of Outdegree")

#fig.colorbar(c, orientation='horizontal')

# Set x-axis label
ax2.set_xlabel('Time (s)')


# Calculate the total number of data points
num_points = out_degree.shape[0]

# Calculate the time step between each data point
dt = window * 1/fs

# Calculate the end time value
end_time = num_points * dt

# Generate an array of time values using numpy.linspace
time_values = np.linspace(0, end_time, num_points + 1)

# Subplot Three
ax3.pcolormesh(np.diff(betweenness_centrality.T), cmap='RdBu_r', shading='gouraud')#, vmin=z_min, vmax=z_max)
divider = make_axes_locatable(ax3)
cax = divider.append_axes("bottom", size="5%", pad=1.5)
fig.colorbar(c, cax=cax, orientation='horizontal')

ax3.set_title("Dervative of Betweenness Centrality")
ax3.set_xlabel('Time (s)')


# Calculate the time array using the same number of data points as out_degree
time = np.linspace(0, out_degree.shape[0] - 2, out_degree.shape[0])

# Plot the line for Average Clustering Coefficient
line1, = ax4.plot(time, avg_clustering_coeffs, label='Average Clustering Coefficient', color='b')
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Average Clustering Coefficient', color='b')
ax4.tick_params(axis='y', labelcolor='b')

# Create a secondary y-axis
axs = ax4.twinx()

# Plot the line for Average Betweenness Centrality
line2, = axs.plot(time, avg_betweenness_centralities, label='Average Betweenness Centrality', color='r')
axs.set_ylabel('Average Betweenness Centrality', color='r')
axs.tick_params(axis='y', labelcolor='r')

# Combine lines and labels for both axes
lines = [line1, line2]
labels = [l.get_label() for l in lines]

# Add the legend to the plot
ax4.legend(lines, labels, loc='upper left')

# Set the x-axis limit to start from 0
ax4.set_xlim(0)

# You can also set the title for the plot
ax4.set_title('Evolution of Topological Features')

# Subplot 5

eeg_data = (x.T - np.mean(x.T))/np.std(x.T)

n_signals = 54


# Plot each EEG signal with an offset
offset = 0
for i in range(n_signals):
    ax5.plot(eeg_data[i] + offset, linewidth=1)
    offset += 2

# Set plot title, labels and limits
ax5.set_title('iEEG Signals')
ax5.set_xlabel('Time')
ax5.set_xlim(0, eeg_data.shape[1])
ax5.set_ylabel('Signal Index')
ax5.set_ylim(-2, n_signals + 4)


# Add the large title to the top of the plot
fig.suptitle('Patient 88', fontsize=24, y=1.02)

# Adjust the spacing between subplots
fig.subplots_adjust(top=0.92)

fig.tight_layout()  # Adjust layout to prevent clipping of labels
plt.setp(ax1.get_xticklabels(), visible=False)
plt.show()

In [ ]:
# Calculate the total number of data points
num_points = x.shape[0]
print("num points", num_points)

# Calculate the time step between each data point
#dt =  * 1/fs
#print("dt", dt)

# Calculate the end time value
end_time = num_points/fs
print("end time", end_time)

# Generate an array of time values using numpy.linspace
time_values = np.linspace(0, end_time, num_points)
print("len time", len(time_values))
print("last time", time_values[-1]) 

In [ ]:
plt.figure(figsize = (13, 9), dpi = 600)
import matplotlib.pyplot as plt
import numpy as np

n_signals = 54
desired_samples = 397  # original number of points (before removing artifact)?

# Define the sampling rate and calculate the downsampling factor
fs = 512  # sampling rate

downsampling_factor = eeg_data.shape[1] // desired_samples
print(downsampling_factor)

# Downsample the data using slicing
downsampled_eeg_data = eeg_data[:, ::downsampling_factor]
#downsampled_eeg_data = downsampled_eeg_data[:, 10:]  # adjust to account for artifact removal
downsampled_eeg_data = downsampled_eeg_data[:, :-1]

# Generate time values based on the downsampling factor
time_values = np.arange(0, downsampled_eeg_data.shape[1]) / fs * downsampling_factor

# Plot the time series
offset = 0
for i in range(n_signals):
    plt.plot(time_values, downsampled_eeg_data[i] + offset, linewidth=1)
    offset += 2


# Add vertical dashed lines at indices with thicker linestyle
plt.axvline(120, color='black', linestyle='--', linewidth=3)
plt.axvline(338, color='black', linestyle='--', linewidth=3)


# Add light grey shading between the vertical lines
#plt.axvspan(80, 99, facecolor=(0.7, 0.7, 0.7), alpha=0.7)

# Draw a grey border around the entire plot
for spine in plt.gca().spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(2)


#plt.title('iEEG Time Series', fontsize=22, pad=20)
plt.xlabel('Time (s)', fontsize=26)
plt.ylabel('Channel', fontsize=26)
#plt.xlim(0, 817)
plt.ylim(1, n_signals + 4)

plt.tick_params(axis='both', labelsize=26)



#plt.savefig('iEEGTimeSeriesQM.jpg', dpi=600, bbox_inches='tight', format='jpeg')
plt.show()

In [ ]:
plt.figure(figsize = (10 , 6), dpi = 600)
node_degree = degree.T  # Assuming degree is defined elsewhere and has shape (num_time_points, num_channels)


print("node_degree :", node_degree.shape)

z_min, z_max = node_degree.min(), np.abs(node_degree).max()

# Plot with the correct dimensions
c = plt.pcolormesh(network_times, range(node_degree.shape[0]), node_degree, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
plt.colorbar(c, orientation='vertical')
plt.axvline(x = 120 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 338 , color = 'black', linestyle = '--', linewidth = 2)


# Set x-axis label
plt.title("Node Degree 88")
plt.xlabel('Time (s)')
plt.ylabel('Channel')
plt.show()

In [ ]:
node_degree.shape

In [ ]:
node_degree_median = np.median(node_degree[: , 30:85], axis = 1) #All ictal 30:84
print(node_degree_median)

In [ ]:
node_degree_median.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(10.2, 6.4), dpi=600)

# Calculate the minimum and maximum values
min_value = np.min(node_degree_median)
max_value = np.max(node_degree_median)

# Normalize data
normalized_node_degree_median = (node_degree_median - min_value) / (max_value - min_value)

# Define the threshold level
threshold = np.percentile(normalized_node_degree_median, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'

# Plot the threshold area
plt.axhspan(threshold, np.max(normalized_node_degree_median), facecolor='lightblue', alpha=0.5)


# Create scatter plot
plt.scatter(x, normalized_node_degree_median, s=60, c=colors)

# Adjusting spine appearance
for spine in plt.gca().spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1.0)

plt.xlabel('Channel', fontsize=20)
plt.ylabel('Median of Node Degree', fontsize=20)
plt.title("Node Degree Criterion", fontsize=22, pad=20)
#plt.legend(loc='upper right', fontsize=10, frameon=True,markerscale=0.8,borderpad=0.3,labelspacing=0.2)

# Display the plot
plt.show()

In [ ]:
np.where(normalized_node_degree_median > threshold) # 8

# Derivative of Node Degree

In [ ]:
# Derivative of Degree QM02
plt.figure(figsize=(10, 6), dpi = 600)

degree_deriv  = np.diff(degree.T) #np.diff(out_degree.T)

yd = degree_deriv
z_min, z_max = yd.min(), np.abs(yd).max()#-np.abs(yd).max(), np.abs(yd).max()
#z_min, z_max = -np.abs(yd).max(), np.abs(yd).max()

c = plt.pcolormesh(network_times[1:], range(yd.shape[0]), yd, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
plt.colorbar(c, orientation='vertical')
plt.axvline(x = 112 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 128, color='black', linestyle='--', linewidth=2.5)



# Set x-axis label
plt.xlabel('Time (s)')
plt.ylabel('Channels')
plt.title("Degree Derivative 88", fontsize=20)



#plt.title("Degree Derivative (${dD}i/dt$)", fontsize=20)
plt.show()

# ABS Max

In [ ]:
degree_abs = np.max(np.abs(degree_deriv[:, 83:85]), axis = 1) # 28(112-124),29(116-128),
print(degree_abs)
degree_abs.shape

In [ ]:
# Normalized scatter plot for NodeDegree
plt.figure(figsize = (10.2,6.5), dpi = 600)

# Calculate the minimum and maximum values 
min_value = np.min(degree_abs)
max_value = np.max(degree_abs)

# Normalize data
normalized_degree_deriv_abs = (degree_abs - min_value) / (max_value - min_value)


# Define the threshold level
threshold = np.percentile(normalized_degree_deriv_abs, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)


# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'



plt.axhspan(threshold, np.max(normalized_degree_deriv_abs), facecolor='lightblue', alpha=0.5)
plt.scatter(x, normalized_degree_deriv_abs, s=60, c=colors)
plt.tick_params(axis='both', which='major', labelsize=20) #18


plt.xlabel('Channel', fontsize = 22)
plt.ylabel('Max Node Degree Derivative' , fontsize = 22)
plt.title("Node Degree Derivative Criterion" , fontsize = 24, pad = 20)

#plt.savefig('NDDerivScatterQM1AbsMax.jpg', dpi=600, bbox_inches='tight', format='jpeg')
plt.show()

In [ ]:
np.where(normalized_degree_deriv_abs > threshold)

# Out_Degree

In [ ]:
# Subplot Two Out Degree
plt.figure(figsize=(10,6), dpi=600)

out_deg = out_degree.T
z_min, z_max = -np.abs(out_deg).max(), np.abs(out_deg).max()


c = plt.pcolormesh(network_times, range(out_deg.shape[0]),out_deg, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
plt.colorbar(c,orientation='vertical')
plt.title("Out Degree")
plt.axvline(x = 120 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 338 , color = 'black', linestyle = '--', linewidth = 2)

plt.xlabel('Time (s)')
plt.ylabel('Channels')
plt.show()

In [ ]:
out_degree_median = np.median(out_deg[: , 30:85], axis = 1) # All ictal 30:84
print(out_degree_median)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4), dpi = 600)

min_value = np.min(out_degree_median)
max_value = np.max(out_degree_median)


normalized_out_deg_median = (out_degree_median - min_value) / (max_value - min_value)

threshold = np.percentile(normalized_out_deg_median, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'

plt.axhspan(threshold, np.max(normalized_out_deg_median), facecolor='lightblue', alpha = 0.5)
plt.scatter(x,normalized_out_deg_median, s=20, c=colors)

plt.xlabel('Channel')
plt.ylabel('Out Degree')
plt.title('Ictal')
plt.show()

In [ ]:
np.where(normalized_out_deg_median > threshold) # none

In [ ]:
# Derivative of Outdegree
plt.figure(figsize = (10,6), dpi = 600)
outdeg_deriv = np.diff(out_degree.T)

z_min, z_max = -np.abs(outdeg_deriv).max(), np.abs(outdeg_deriv).max()
c = plt.pcolormesh(network_times[:-1], range(outdeg_deriv.shape[0]), outdeg_deriv, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
plt.colorbar(c,  orientation='vertical')
plt.title("Outdegree Derivative")
plt.axvline(x = 112 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 128, color='black', linestyle='--', linewidth=2.5)

# Set x-axis label
plt.xlabel('Time (s)')
plt.show()

In [ ]:
outdeg_abs = np.max(np.abs(outdeg_deriv[:, 29:31]), axis = 1) # 29:30
#print(outdeg_abs)
outdeg_abs.shape

In [ ]:
plt.figure(figsize=(10, 6), dpi=600)


# Calculate the minimum and maximum values
min_value = np.min(outdeg_abs)
max_value = np.max(outdeg_abs)

# Normalize using the new formula
normalized_outdegree_abs = (outdeg_abs - min_value) / (max_value - min_value)

# Define the threshold level
threshold = np.percentile(normalized_outdegree_abs, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'


plt.axhspan(threshold, np.max(normalized_outdegree_abs), facecolor='lightblue', alpha=0.5)
plt.scatter(x, normalized_outdegree_abs, s=60, c=colors)

    
plt.xlabel('Channel', fontsize=20)
plt.ylabel('Out Degree Derivative', fontsize=20)
plt.title('Maximum of Out Degree Derivative' , fontsize = 22, pad = 20)

#plt.savefig('*OutDDrIctalScatterQM3.jpg', dpi=600, bbox_inches='tight', format='jpeg')
plt.show()

In [ ]:
np.where(normalized_outdegree_abs > threshold) # none

# In Degree

In [ ]:
plt.figure(figsize=(10, 6), dpi=600)

in_deg = in_degree.T
z_min, z_max = -np.abs(in_deg).max(), np.abs(in_deg).max()

c = plt.pcolormesh(network_times, range(in_deg.shape[0]), in_deg, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
plt.colorbar(c, orientation='vertical')
plt.axhline(18, color = 'black', linewidth = 1, linestyle = "--")
plt.title("In Degree")
plt.axvline(x = 120 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 338 , color = 'black', linestyle = '--', linewidth = 2)


plt.xlabel('Time (s)')
plt.ylabel('Channel')
plt.show()

In [ ]:
in_degree_median = np.median(in_deg[: , 30:85], axis = 1) # All ictal 30:84
print(in_degree_median)

In [ ]:
plt.figure(figsize=(6, 4), dpi = 600)


min_value = np.min(in_degree_median)
max_value = np.max(in_degree_median)


normalized_in_deg_median = (in_degree_median - min_value) / (max_value - min_value)

threshold = np.percentile(normalized_in_deg_median, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'



plt.axhspan(threshold, np.max(normalized_in_deg_median), facecolor='lightblue', alpha=0.5)
plt.scatter(x,normalized_in_deg_median, s=20, c=colors)

plt.xlabel('Channel', fontsize=14)
plt.ylabel('In Degree', fontsize=14)
#plt.title('Ictal')

#plt.savefig('InDegScatterQM1median.jpg', dpi=600, bbox_inches='tight', format='jpeg')
plt.show()

In [ ]:
np.where (normalized_in_deg_median > threshold) # 8

In [ ]:
# Derivative of In Degree
plt.figure(figsize=(10, 6), dpi=600)

indeg_deriv = np.diff(in_degree.T)

z_min, z_max = -np.abs(indeg_deriv).max(), np.abs(indeg_deriv).max()

c = plt.pcolormesh(network_times[:-1], range(indeg_deriv.shape[0]), indeg_deriv, cmap='RdBu_r', shading='gouraud', vmin=z_min, vmax=z_max)
plt.colorbar(c, orientation='vertical')
plt.title("Derivative of InDegree")
plt.axvline(x = 112 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 128, color='black', linestyle='--', linewidth=2.5)



# Set x-axis label
plt.xlabel('Time (s)')
plt.ylabel('Channel')
plt.show()

In [ ]:
indegree_abs = np.max(np.abs(indeg_deriv[:, 29:31]), axis = 1) # 29:30
#print(indegree_abs)
indegree_abs.shape

In [ ]:
# Normalized scatter plot for NodeDegree
plt.figure(figsize = (10.2,6.5), dpi = 600)

# Calculate the minimum and maximum values 
min_value = np.min(indegree_abs)
max_value = np.max(indegree_abs)

# Normalize data
normalized_indegree_deriv_abs = (indegree_abs - min_value) / (max_value - min_value)


# Define the threshold level
threshold = np.percentile(normalized_indegree_deriv_abs, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'


plt.axhspan(threshold, np.max(normalized_indegree_deriv_abs), facecolor='lightblue', alpha=0.5)
plt.scatter(x, normalized_indegree_deriv_abs, s=60, c=colors)
plt.tick_params(axis='both', which='major', labelsize=20) #18


plt.xlabel('Channel', fontsize = 22)
plt.ylabel('Max In Degree Derivative' , fontsize = 22)
plt.title("In Degree Derivative Criterion" , fontsize = 24, pad = 20)

#plt.savefig('InDerivScatterQM1AbsMax.jpg', dpi=600, bbox_inches='tight', format='jpeg')
plt.show()


In [ ]:
np.where(normalized_indegree_deriv_abs > threshold) # no purple/green

# Betweenness centrality

In [ ]:
# Betweenness Centrality
plt.figure(figsize=(10,6), dpi=600)

bc = betweenness_centrality.T
z_min, z_max = bc.min(), np.abs(bc).max()

plt.pcolormesh(network_times,range(bc.shape[0]), bc, cmap='RdBu_r', shading='gouraud' ,vmin=z_min, vmax=z_max)
fig.colorbar(c, orientation='vertical')
#plt.axhline(18, color = 'white', linestyle = '--', linewidth = 2)
plt.axvline(x = 120 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 338 , color = 'black', linestyle = '--', linewidth = 2)

plt.title("Betweenness Centrality")
plt.xlabel('Time (s)')
plt.show()

In [ ]:
bc_median = np.median(bc[: , 30:85], axis = 1) # All ictal 30:84
print(bc_median.shape)

In [ ]:
plt.figure(figsize=(6, 4), dpi = 600)


min_value = np.min(bc_median)
max_value = np.max(bc_median)


normalized_bc_median= (bc_median - min_value) / (max_value - min_value)

threshold = np.percentile(normalized_bc_median, 95)


# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'


plt.axhspan(threshold, np.max(normalized_bc_median),facecolor='lightblue', alpha = 0.5)
plt.scatter(x, normalized_bc_median, s=20, c=colors)

plt.xlabel('Channel')
plt.ylabel('Betweeness Centrality')
plt.show()

In [ ]:
np.where(normalized_bc_median > threshold)  # 8(purple)

In [ ]:
#Derivative of Betweenness Centrality
plt.figure(figsize=(6, 4), dpi = 600) 
bc_deriv = np.diff(betweenness_centrality.T)
z_min, z_max = bc_deriv.min(), np.abs(bc_deriv).max()

plt.pcolormesh(network_times[:-1],range(bc_deriv.shape[0]), bc_deriv, cmap='RdBu_r', shading='gouraud' ,vmin=z_min, vmax=z_max)
plt.colorbar(c, orientation='vertical')
#plt.axhline(46, color = 'white', linestyle = '--', linewidth = 2)
plt.axvline(x = 112 , color = 'black', linestyle = '--', linewidth = 2)
plt.axvline(x = 128, color='black', linestyle='--', linewidth=2.5)


plt.title("Derivative of Betweenness Centrality")
plt.xlabel('Time (s)')
plt.show()

In [ ]:
bc_abs = np.max(np.abs(bc_deriv[:, 29:31]), axis = 1) # 29:30
print(bc_abs)
bc_abs.shape

In [ ]:
plt.figure(figsize=(6, 4), dpi = 600)


min_value = np.min(bc_abs)
max_value = np.max(bc_abs)
normalized_bc_deriv_abs= (bc_abs - min_value) / (max_value - min_value)

threshold = np.percentile(normalized_bc_deriv_abs, 95)

# Existing red channels
# resect, SOZ
resect_SOZ = [18]
SOZ = list(range(16, 18)) + list(range(26, 29)) + list(range(45, 47))
resect = list(range(3, 9))

# Convert to sets for easier comparison
resect_soz_set = set(resect_SOZ) # green
soz_set = set(SOZ)  # red
resect_set = set(resect) # purple


print('Resect SOZ:', resect_SOZ)
print('SOZ:', SOZ)
print('Resect:', sorted(resect))

x = np.linspace(0, 54, 54)

# Create an array to store colors, initialized to 'steelblue'
colors = ['steelblue'] * len(x)

# Set the color to 'red' for soz
for channel in soz_set:
    if 0 <= channel < len(x):
        colors[channel] = 'red'

# Set the color to 'green' for resect and soz
for node in resect_SOZ:
    if 0 <= node < len(x):
        colors[node] = 'green'

# Set the color to 'purple' for resect
for node in resect:
    if 0 <= node < len(x):
        colors[node] = 'purple'

plt.axhspan(threshold, np.max(normalized_bc_deriv_abs),facecolor='lightblue', alpha = 0.5)
plt.scatter(x,normalized_bc_deriv_abs, s=10, c=colors)

plt.xlabel('Channel')
plt.ylabel('Betweeness Centrality')
plt.title('Ictal')
plt.show()

In [ ]:
np.where(normalized_bc_deriv_abs > threshold) # no purple or green